In [3]:
# !pip install -U raydp

In [1]:
import pyspark; print(f'pyspark=={pyspark.__version__}')
import ray; print(f'ray=={ray.__version__}')
import raydp; print(f'raydp=={raydp.__version__}')
import pandas; print(f'pandas=={pandas.__version__}')


pyspark==3.2.0
ray==1.9.1
raydp==0.4.1
pandas==1.1.4


In [2]:
!java --version

openjdk 11.0.13 2021-10-19
OpenJDK Runtime Environment (build 11.0.13+8-Ubuntu-0ubuntu1.20.04)
OpenJDK 64-Bit Server VM (build 11.0.13+8-Ubuntu-0ubuntu1.20.04, mixed mode, sharing)


In [3]:
from hyperplane.ray_common import initialize_ray_cluster, stop_ray_cluster, find_ray_workers
num_workers = 1
cpu_core_per_worker = 15
ram_gb_per_worker = 12 #110 GB allocatible for 16_128 nodes, 12 for 16_16 nodes, 27 for 32_32 nodes
ray_cluster = initialize_ray_cluster(num_workers, cpu_core_per_worker, ram_gb_per_worker)

👉 Hyperplane: selecting worker node pool


2021-12-29 19:34:17,208	INFO services.py:1338 -- View the Ray dashboard at http://10.2.39.2:8787
2021-12-29 19:34:17,214	WARNING services.py:1816 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=8.59gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


ray dashboard available at https://shakdemo.hyperplane.dev/ray-stella2/#/
Waiting for worker ray-worker-faea11e5-338d-4f3c-a49e-462cf6b9cdb5...


In [4]:
from pyspark import SparkContext, SparkConf
# conf = SparkConf().set('spark.ui.port', '8787')
sc = SparkContext()
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/29 19:38:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark = raydp.init_spark('example', num_executors=2, executor_cores=4, executor_memory='4G')

2021-12-29 19:38:54,646 WARN NativeCodeLoader [Thread-1]: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-12-29 19:38:54,756 INFO SecurityManager [Thread-1]: Changing view acls to: root
2021-12-29 19:38:54,757 INFO SecurityManager [Thread-1]: Changing modify acls to: root
2021-12-29 19:38:54,758 INFO SecurityManager [Thread-1]: Changing view acls groups to: 
2021-12-29 19:38:54,759 INFO SecurityManager [Thread-1]: Changing modify acls groups to: 
2021-12-29 19:38:54,759 INFO SecurityManager [Thread-1]: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(root); groups with view permissions: Set(); users  with modify permissions: Set(root); groups with modify permissions: Set()
2021-12-29 19:38:55,146 INFO Utils [Thread-1]: Successfully started service 'RAY_RPC_ENV' on port 34967.


In [9]:
import pyspark.pandas as pd

In [10]:
df = pd.read_csv("/root/data_nps_score_data_NPS.csv")
df.shape

(29700, 25)

In [11]:
df.head(2)

,Response Date,Unique ID,Survey Type,Country,Unit,Industry,Business Unit,Project ID,Source Type,LTR Services,LTC Services,Overall Satisfaction,Communication,Expertise,Industry Expertise,Innovation,Partnership,Proactivity,Quality,Responsiveness,Staffing,Technical Expertise,Timeliness,Value,Reminded
0,2020-07-11 19:55:35,UIDRS5770.26768582561,Type2,JP,Company976,Chemical,Gateway,PRUIDRS5770.2676858256119,Telephonic,2,2,2.0,8,9,8,7,8,5,8,9,9,8,8,8.0,No
1,2020-07-11 19:42:00,UIDRS8653.02010100842,Type2,EC,Company351,Financial,Web,PRUIDRS8653.0201010084219,Mail,8,8,8.0,8,8,8,7,8,10,8,8,9,8,8,8.0,No


In [12]:
df.groupby('Source Type').size()

Source Type
Telephonic     9663
Mail           2056
Online        17981
dtype: int64

In [13]:
%%time
df.Value.mean()

CPU times: user 20.7 ms, sys: 2.47 ms, total: 23.2 ms
Wall time: 325 ms


8.156164664117968

In [26]:
## make plot show inline on jlab
import plotly.io as pio
pio.renderers.default = 'iframe'

In [27]:
%%time
import numpy as np
pyspark.pandas.DataFrame(np.random.rand(100, 4), columns=list("abcd")).plot.area()

CPU times: user 115 ms, sys: 15.1 ms, total: 130 ms
Wall time: 378 ms


In [28]:
stop_ray_cluster(ray_cluster)

Deleting ray-worker-faea11e5-338d-4f3c-a49e-462cf6b9cdb5
